In [ ]:
import pandas as pd
import numpy as np
import sweetviz as sv
import ipaddress

from scapy.all import PcapReader, IP, TCP, UDP, ICMP
from scipy.stats import ttest_ind, kstest, norm, skew, kurtosis, zscore
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

from skimpy import skim
from summarytools import dfSummary
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [ ]:
pcap_reader_mirai = PcapReader("data/mirai.pcap")
pcap_reader_benign = PcapReader("data/benign.pcapng")

In [ ]:
type(pcap_reader_benign)

# Preprocess

- convert data to streams
- collect some numbers

In [ ]:
def pcap_to_dataframe(pcap_reader: PcapReader) -> pd.DataFrame:
    """_summary_

    Args:
        pcap_reader (PcapReader): packet capture read using scapy

    Returns:
        pd.DataFrame: dataframe with pcap data
    """
    # Create an empty list to store the data
    data = []

    # Iterate through the packets in the pcap file
    for packet in pcap_reader:
        # Get the source and destination IP addresses
        if packet.haslayer(IP):
            src_ip = packet[IP].src
            dst_ip = packet[IP].dst
            protocol = packet[IP].proto
        else:
            src_ip = None
            dst_ip = None
            protocol = None

        # Get the source and destination ports and payload
        if packet.haslayer(TCP):
            src_port = packet[TCP].sport
            dst_port = packet[TCP].dport
            payload = str(packet[TCP].payload)
            packet_len = len(packet[TCP])
        elif packet.haslayer(UDP):
            src_port = packet[UDP].sport
            dst_port = packet[UDP].dport
            payload = str(packet[UDP].payload)
            packet_len = len(packet[UDP])
        elif packet.haslayer(ICMP):
            payload = str(packet[ICMP].payload)
            packet_len = len(packet[ICMP])
            src_port = None
            dst_port = None
        else:
            src_port = None
            dst_port = None
            payload = str(packet.payload)
            packet_len = len(packet)

        # Append the data to the list
        data.append(
            [
                packet.time,
                src_ip,
                dst_ip,
                src_port,
                dst_port,
                payload,
                packet_len,
                protocol,
            ]
        )

    # Convert the list to a pandas dataframe
    df = pd.DataFrame(
        data,
        columns=[
            "Timestamp",
            "Source IP",
            "Destination IP",
            "Source Port",
            "Destination Port",
            "Payload",
            "Packet Length",
            "Protocol",
        ],
    )

    return df

In [ ]:
# mirai_df = pcap_to_dataframe(pcap_reader_mirai)
# benign_df = pcap_to_dataframe(pcap_reader_benign)

In [ ]:
# mirai_df.to_pickle("../data/bsides_aug/mirai.pkl")
# benign_df.to_pickle("../data/bsides_aug/benign.pkl")

In [ ]:
mirai_df = pd.read_pickle("data/mirai.pkl")
benign_df = pd.read_pickle("data/benign.pkl")

In [ ]:
mirai_df

In [ ]:
benign_df

In [ ]:
# TODO: use pandas AI to ask questions from the dataframe

In [ ]:
def extract_streams(df: pd.DataFrame) -> pd.DataFrame:
    # Create an empty list to store stream data as separate dataframes
    dfs = []

    # Group packets by src/dst IP and src/dst port
    grouped = df.groupby(
        ["Source IP", "Destination IP", "Source Port", "Destination Port", "Protocol"]
    )

    # Iterate through each group to extract stream data
    for name, group in grouped:
        # Get source/destination IP, port, and protocol
        src_ip, dst_ip, src_port, dst_port, proto = name

        # Get number of packets, total length, and duration of the stream
        num_packets = len(group)
        total_length = group["Packet Length"].sum()
        start_time = group["Timestamp"].min()
        end_time = group["Timestamp"].max()
        duration = float(end_time - start_time)

        # Create a new dataframe with the stream data
        stream_df = pd.DataFrame(
            {
                "Source IP": [src_ip],
                "Destination IP": [dst_ip],
                "Source Port": [src_port],
                "Destination Port": [dst_port],
                "Protocol": [proto],
                "Number of Packets": [num_packets],
                "Total Length": [total_length],
                "Duration": [duration],
            }
        )

        # Add the new dataframe to the list
        dfs.append(stream_df)

    # Concatenate all the dataframes in the list into one dataframe
    stream_df = pd.concat(dfs, ignore_index=True)

    # Return the new dataframe with stream data
    return stream_df

In [ ]:
mirai_stream_df = extract_streams(mirai_df)
benign_stream_df = extract_streams(benign_df)

# EDA

Exploratory Data Analysis approaches the dataset as a black box that we need to visualize and analyze statistically with the following goals:
- get insights about our data
- test hypotheses
- decide on models and further processing, such as feature engineering.

EDA can be performed for benign and malicious data. Here we are looking at EDA only for malicious data, however the same functions can be applied to benign.

## Descriptive statistics & data

- Describe columns and data types
- Descriptive statistics
  -  count, 
  -  mean, 
  -  standard deviation, 
  -  minimum, 
  -  25th percentile, 
  -  median (50th percentile), 
  -  75th percentile, and 
  -  maximum

In [ ]:
# describe, summarize etc.
mirai_stream_df.columns

In [ ]:
mirai_stream_df.dtypes

In [ ]:
# descriptive statistics
mirai_stream_df.describe()

In [ ]:
# correlation matrix for numerical values in dataframe
mirai_stream_df.corr()

## Hypothesis testing

- Is the difference between two groups or variables statistically significant?
- Use t-test to compare means of two groups
  - assumes that data follows normal distribution
- Types of variables
  - dependent: the effect of a phenomenon. For example, how does number of HTTP requests mean that a network is compromised?
  - independent: the cause. The number of HTTP requests affects whether a network is compromised.

In [ ]:
def hypothesis_testing(df, col1, col2):
    group1 = df[col1]
    group2 = df[col2]
    pvalue = ttest_ind(group1, group2)[1]
    if pvalue < 0.05:
        return "The difference between {} and {} is statistically significant (p < 0.05)".format(
            col1, col2
        )
    else:
        return "The difference between {} and {} is not statistically significant (p >= 0.05)".format(
            col1, col2
        )

In [ ]:
hypothesis_testing(mirai_stream_df, "Number of Packets", "Total Length")

## Regression Analysis

- Models relationship between a dependent variable and one or more independent variables
- Linear regression
  - fit data in line
  - calculate coefficients

In [ ]:
def regression_analysis(df, x_cols, y_col):
    X = df[x_cols].values.reshape(-1, len(x_cols))
    y = df[y_col].values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    r_sq = model.score(X, y)
    coef = model.coef_
    return {"R-squared": r_sq, "Coefficients": coef}

In [ ]:
regression_analysis(mirai_stream_df, ["Number of Packets", "Duration"], "Total Length")

## Kolmogorov-Smirnov test

- compare two sample distributions
- useful for fitting to a distribution
- test if two samples from a population:
  - came from a distribution
  - belong to the same distribution
- Uses metric `D`
  - max absolute difference between empirical distribution function of the samples and cumulative distribution of the reference distribution
- Null hypothesis: 
  - samples came from the reference distribution
  - samples came from the same distribution

In [ ]:
def kolmogorov_smirnov_test(df, column):
    sample = df[column].values
    _, pvalue = kstest(sample, norm.cdf, args=(sample.mean(), sample.std()))
    if pvalue < 0.05:
        return "The distribution of {} is significantly different from a normal distribution (p < 0.05)".format(
            column
        )
    else:
        return "The distribution of {} is not significantly different from a normal distribution (p >= 0.05)".format(
            column
        )

In [ ]:
kolmogorov_smirnov_test(mirai_stream_df, "Total Length")

## Skewness and Kyrtosis

- information about the shape of the distribution
- Skewness: measure the degree of asymmetry
  - symmetric: equally balanced around its mean
  - asymmetric: not equally balanced
  - positive skewness: distribution longer on the right side
  - negative skewness: longer on the left
  - 0: completely symmetric
- Kurtosis: peakedness of distribution
  - high: sharp peak, long tails
  - low: flat peak, short tails
  - ex. normal distribution has kurtosis 3, mesokurtic
    - `> 3` leptokurtic
    - `< 3` platykurtic

In [ ]:
def skewness_kurtosis(df):
    result = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        result[col + "_skewness"] = skew(df[col])
        result[col + "_kurtosis"] = kurtosis(df[col])
    return result

In [ ]:
skewness_kurtosis(mirai_stream_df)

## Outliers

- observation that significantly differs from others in a dataset
- Causes
  - measurement errors
  - extreme rare values
- significant impact in statistical analysis
- measurements
  - z-score: `(x - mean) / std_dev`
  - IQR method: this method identifies outliers as observations that are below `Q1 - 1.5IQR` or above `Q3 + 1.5IQR`, where Q1 and Q3 are the first and third quartiles, and IQR is the interquartile range (the difference between Q3 and Q1).
  - visual inspection

In [ ]:
def detect_outliers_zscore(df, column, threshold=3):
    zscores = np.abs(zscore(df[column]))
    return df[zscores > threshold]

In [ ]:
outliers = detect_outliers_zscore(mirai_stream_df, "Total Length", threshold=3)
print(outliers)

# Feature Engineering

## Numerical

In [ ]:
# convert ip address to numeric values
def ip_to_numeric(ip):
    ip_obj = ipaddress.ip_interface(ip)
    return int(ip_obj.network.network_address)

In [ ]:
# convert IPs to numeric mirai data
mirai_stream_df["Source IP Numeric"] = mirai_stream_df["Source IP"].apply(ip_to_numeric)
mirai_stream_df["Destination IP Numeric"] = mirai_stream_df["Destination IP"].apply(
    ip_to_numeric
)

In [ ]:
# convert IPs to numeric benign data
benign_stream_df["Source IP Numeric"] = benign_stream_df["Source IP"].apply(
    ip_to_numeric
)
benign_stream_df["Destination IP Numeric"] = benign_stream_df["Destination IP"].apply(
    ip_to_numeric
)

In [ ]:
# get rid of non numeric columns for IPs
mirai_stream_df_numeric = mirai_stream_df.drop(columns=["Source IP", "Destination IP"])
benign_stream_df_numeric = benign_stream_df.drop(
    columns=["Source IP", "Destination IP"]
)

In [ ]:
# convert duration from object to float
mirai_stream_df["Duration"] = mirai_stream_df_numeric["Duration"].astype(float)
benign_stream_df["Duration"] = benign_stream_df_numeric["Duration"].astype(float)

In [ ]:
# check if all data types are numeric now
mirai_stream_df_numeric.dtypes

# Summaries & Visualizations

In [ ]:
skim(mirai_stream_df_numeric)

In [ ]:
skim(benign_stream_df_numeric)

In [ ]:
dfSummary(mirai_stream_df_numeric)

In [ ]:
dfSummary(benign_stream_df_numeric)

In [ ]:
my_report = sv.analyze(mirai_stream_df_numeric)
my_report.show_html()

In [ ]:
my_report = sv.analyze(benign_stream_df_numeric)
my_report.show_html()

# Labeling
We label and concatenate benign and malicious before one-hot because there are different ports in each dataset and concatenating the two after one hot will not work with different columns.

In [ ]:
# add labels, 0 for benign, 1 for malicious
mirai_stream_df_numeric["Labels"] = 1
benign_stream_df_numeric["Labels"] = 0

In [ ]:
# concatenate dataframes
data_df = pd.concat([mirai_stream_df_numeric, benign_stream_df_numeric], ignore_index=True)

# Feature engineering (cont.) categorical 
## One-hot

In [ ]:
def one_hot_encode(df, feature):
    feature_dummies = pd.get_dummies(df[feature], prefix=feature)
    return pd.concat([df, feature_dummies], axis=1)

In [ ]:
data_df = one_hot_encode(data_df, "Source Port")
data_df = one_hot_encode(data_df, "Destination Port")

In [ ]:
# TODO: use skllm or embeddings

# Model Training
- models
  - xgboost
  - NN
  - k-NN
  - Random Forest
- k-fold cross validation

In [ ]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=1)

In [ ]:
# randomize the data. TODO: is this needed
from sklearn.utils import shuffle
data_df = shuffle(data_df)

In [ ]:
# after feature engineering, drop all the columns that are unnecessary
data_df = data_df.drop(["Source Port", "Destination Port"], axis=1)

In [ ]:
# store the features and labels in separate dataframes
X = data_df.drop("Labels", axis=1)
y = data_df["Labels"]

In [ ]:
# convert data to array
data_array = data_df.to_numpy()

In [ ]:
model = 1
for train, test in kfold.split(data_array):
    print(f"Model {model}")
    print(f"train {train}, test: {test}")
    model += 1

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')

In [ ]:
def train_evaluate_model(model, X, y):
    accuracy_scores = []
    for train, test in kfold.split(X):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y.iloc[train], y.iloc[test]
        
        model.fit(X_train, y_train)
        
        # evaluate model accuracy
        accuracy = model.score(X_test, y_test)
        accuracy_scores.append(accuracy)
        
    return accuracy_scores

# Model Evaluation

In [ ]:
# Calculate the average accuracy score across all folds
accuracy_scores = train_evaluate_model(knn_model, X, y)
average_accuracy = sum(accuracy_scores) / k
print("Average accuracy k-nn:", average_accuracy)

In [ ]:
# Calculate the average accuracy score across all folds
accuracy_scores = train_evaluate_model(rf_model, X, y)
average_accuracy = sum(accuracy_scores) / k
print("Average accuracy random forest:", average_accuracy)

In [ ]:
# Calculate the average accuracy score across all folds
accuracy_scores = train_evaluate_model(xgb_model, X, y)
average_accuracy = sum(accuracy_scores) / k
print("Average accuracy XGBoost:", average_accuracy)